# Tutorial V - Economic Dispatch Problem

Energy System Optimization with Julia

# 1. Modelling the ED problem

Implement the ED problem from the lecture in Julia. Before we start,
let’s load the necessary packages and data.

In [1]:
using JuMP, HiGHS
using CSV
#using DelimitedFiles
using DataFrames
using Plots
#using StatsPlots
#import Pkg; Pkg.add("PlotlyBase")
#plotly() # This will create interactive plots later on

> **Tip**
>
> If you haven’t installed the packages yet, you can do so by running
> `using Pkg` first and then `Pkg.add("JuMP")`, `Pkg.add("HiGHS")`,
> `Pkg.add("DataFrames")`, `Pkg.add("Plots")`, and
> `Pkg.add("StatsPlots")`.

Now, let’s load the data. The generator data
($p^{\min}_g, p^{\max}_g, c^{var}_g, c^{fix}_g$), $c^{fix}_g$ being
fixed cost not used in the ED, the wind data ($c^{var}_w$), and the
scenario data ($p^f_w, d^f$) are provided as CSV files.

In [1]:
# Get the directory of the current file
file_directory = "$(@__DIR__)/data"

# Load the data of the thermal generators
generators = CSV.read("$file_directory/generator.csv", DataFrame)
println("Number of generator: $(nrow(generators))")
println("First 5 rows of available genrator:")
println(generators[1:5, :])

Number of generator: 6
First 5 rows of available genrator:
5×5 DataFrame
 Row │ name     min_power  max_power  variable_cost  fix_cost 
     │ String3  Int64      Int64      Int64          Int64    
─────┼────────────────────────────────────────────────────────
   1 │ G1             100        500             50      1000
   2 │ G2              50        350             60      1200
   3 │ G3              40        250             55      1300
   4 │ G4              30        200             70      1500
   5 │ G5              30        200             60      1500

In [10]:
# Load the data of the wind turbines
windTurbines = CSV.read("$file_directory/windTurbine.csv", DataFrame)
println("Number of wind turbines: $(nrow(windTurbines))")
println("Variable cost per wind turbine:")
println(windTurbines)

Number of wind turbines: 1
Variable cost per wind turbine:
1×2 DataFrame
 Row │ name     var_cost 
     │ String3  Int64    
─────┼───────────────────
   1 │ T1             50


In [11]:
# Load the sceanrio data about the demand and wind forecast
scenarios = CSV.read("$file_directory/scenario.csv", DataFrame)
println("First 5 rows of sceanios:")
println(scenarios[1:5, :])

First 5 rows of sceanios:
5×3 DataFrame
 Row │ name     wind_forecast  demand_forecast 
     │ String3  Int64          Int64           
─────┼─────────────────────────────────────────
   1 │ S1                1000             1500
   2 │ S2                1000             1600
   3 │ S3                1000             1400
   4 │ S4                1000             1300
   5 │ S5                1000             1000


Next, you need to prepare the given data for the model. We will use
‘function’ to create a ‘Named Tuple’ which we can access with the dot
notation:

In [12]:
# This function creates the Named Tuple ThermalGenerator
function ThermalGenerator(
    min::Int64,
    max::Int64,
    fixed_cost::Int64,
    variable_cost::Int64,
)
    return (
        min = min,
        max = max,
        fixed_cost = fixed_cost,
        variable_cost = variable_cost,
    )
end

# Add generators of the data to a dictionary of the generators
dictThermalGeneartors = Dict(row.name => ThermalGenerator(row.min_power, row.max_power, row.fix_cost, row.variable_cost) for row in eachrow(generators))

# Now a generator propety can be accessed
println(dictThermalGeneartors["G1"].variable_cost)

50


Analogously create a dictionary for the wind turbines and scenarios.
Call them `dictWindTurbines` and `dictScenarios`.

In [ ]:
# YOUR CODE BELOW
# This function creates the Named Tuple WindGenerator
function WindTurbine(
    variable_cost::Int64,
)
    return (
        variable_cost = variable_cost,
    )
end
# This function creates the Named Tuple Scenario
function Scenario(
    wind_forecast::Int64,
    demand_forecast::Int64,
)
    return (
        wind_forecast = wind_forecast,
        demand_forecast = demand_forecast,
    )
end

# Add generators of the data to a dictionary of the dict
dictWindTurbines = Dict(row.name => WindTurbine(row.var_cost) for row in eachrow(windTurbines))
dictScenarios = Dict(row.name => Scenario(row.wind_forecast, row.demand_forecast) for row in eachrow(scenarios))

# Now a generator propety can be accessed
println(dictWindTurbines["T1"].variable_cost)
println(dictScenarios["S1"].wind_forecast)


50
1000


In [21]:
# Validate your solution
@assert length(dictThermalGeneartor) == nrow(generators) "Available time dictionary should have same length as input data"
@assert length(dictWindTurbines) == nrow(windTurbines) "Available time dictionary should have same length as input data"
@assert length(dictScenarios) == nrow(scenarios) "Bottling time dictionary should have same length as input data"

# Check that all values are positive
@assert all(v -> all(x -> x >= 0, [v.min, v.max, v.fixed_cost, v.variable_cost]), values(dictThermalGeneartor)) "All thermal generator values must be positive"
@assert all(v -> v.variable_cost >= 0, values(dictWindTurbines)) "All wind turbine values must be positive"
@assert all(v -> all(x -> x >= 0, [v.wind_forecast, v.demand_forecast]), values(dictScenarios)) "All scenario values must be positive"

# Check that dictionaries contain all expected keys
@assert all(p -> haskey(dictThermalGeneartor, p), generators.name) "Missing names in dictionary"
@assert all(b -> haskey(dictWindTurbines, b), windTurbines.name) "Missing names in dictionary"
@assert all(b -> haskey(dictScenarios, b), scenarios.name) "Missing names in dictionary"

Next, we define the model instance for the ED problem.

In [22]:
# Prepare the model instance
dispatchModel = Model(HiGHS.Optimizer)


A JuMP Model
├ solver: HiGHS
├ objective_sense: FEASIBILITY_SENSE
├ num_variables: 0
├ num_constraints: 0
└ Names registered in the model: none

Now, create your variables. Please name them `p_g` for the power output
of generators, `p_w` for the power injection of wind turbines.

> **Note**
>
> Consider the bounds for these variables. First, we only want to solve
> the model for sceanrio “S1”.

In [29]:
# YOUR CODE BELOW
# Create variables for thermal generator power output with bounds
p_g = @variable(dispatchModel, [g in keys(dictThermalGeneartor)], 
    lower_bound = dictThermalGeneartor[g].min,
    upper_bound = dictThermalGeneartor[g].max,
    base_name = "p_g")

# Create variables for wind turbine power output with bounds
# Wind power is bounded by the forecast for scenario S1
p_w = @variable(dispatchModel, [w in keys(dictWindTurbines)],
    lower_bound = 0,
    upper_bound = dictScenarios["S1"].wind_forecast,
    base_name = "p_w")



1-dimensional DenseAxisArray{VariableRef,1,...} with index sets:
    Dimension 1, String3["T1"]
And data, a 1-element Vector{VariableRef}:
 p_w[T1]

In [32]:
# Validate your solution
# Check if variables exist in the model
#@assert haskey(dispatchModel.obj_dict, :p_g) "p_g variable not found in model"
#@assert haskey(dispatchModel.obj_dict, :p_w) "p_w variable not found in model"

# Check variable dimensions
@assert length(p_g) == length(dictThermalGeneartor) "Incorrect dimensions for p_g"
@assert length(p_w) == length(dictWindTurbines) "Incorrect dimensions for p_w"

# Check variable types
@assert all(x -> is_valid(dispatchModel, x), p_g) "p_g must be valid variables"
@assert all(x -> is_valid(dispatchModel, x), p_w) "p_w must be valid variables"

Next, define the objective function.

In [33]:
# YOUR CODE BELOW
# Define objective function: minimize total generation costs
@objective(dispatchModel, Min,
    # Thermal generator costs
    sum(dictThermalGeneartor[g].variable_cost * p_g[g] for g in keys(dictThermalGeneartor)) +
    # Wind turbine costs  
    sum(dictWindTurbines[w].variable_cost * p_w[w] for w in keys(dictWindTurbines))
)


55 p_g[G3] + 60 p_g[G5] + 70 p_g[G4] + 50 p_g[G1] + 60 p_g[G2] + 100 p_g[G6] + 50 p_w[T1]

In [34]:
# Validate your solution
# Check if the model has an objective
@assert objective_function(dispatchModel) !== nothing "Model must have an objective function"

# Check if it's a minimization problem
@assert objective_sense(dispatchModel) == MOI.MIN_SENSE "Objective should be minimization"

# Check if the objective function contains both cost components
obj_expr = objective_function(dispatchModel)
@assert contains(string(dispatchModel), "p_g") "Objective must include variable costs (p_g)"
@assert contains(string(dispatchModel), "p_w") "Objective must include variable costs (p_w)"

Now, we need to define all necessary constraints for the model, which is
only the demand/production balance constraint as we considered min and
max power limitations in the variable setup.

In [38]:
# YOUR CODE BELOW
# Power balance constraint: total generation equals demand for scenario S1
@constraint(dispatchModel, 
    sum(p_g[g] for g in keys(dictThermalGeneartor)) + 
    sum(p_w[w] for w in keys(dictWindTurbines)) == 
    dictScenarios["S1"].demand_forecast
)


p_g[G3] + p_g[G5] + p_g[G4] + p_g[G1] + p_g[G2] + p_g[G6] + p_w[T1] = 1500

Finally, implement the solve statement for your model instance and print
the results.

In [39]:
# YOUR CODE BELOW
# Solve the model
optimize!(dispatchModel)

# Print results
println("Optimal objective value: EUR ", objective_value(dispatchModel))
println("\nGenerator dispatch (MW):")
for g in keys(dictThermalGeneartor)
    println("$g: ", value(p_g[g]))
end

println("\nWind turbine dispatch (MW):")
for w in keys(dictWindTurbines)
    println("$w: ", value(p_w[w]))
end

println("\nWind curtailment (MW):")
for w in keys(dictWindTurbines)
    println("$w: ", dictScenarios["S1"].wind_forecast - value(p_w[w]))
end


Running HiGHS 1.9.0 (git hash: 66f735e60): Copyright (c) 2024 HiGHS under MIT licence terms
Coefficient ranges:
  Matrix [1e+00, 1e+00]
  Cost   [5e+01, 1e+02]
  Bound  [3e+01, 1e+03]
  RHS    [2e+03, 2e+03]
Presolving model
1 rows, 7 cols, 7 nonzeros  0s
1 rows, 5 cols, 5 nonzeros  0s
0 rows, 0 cols, 0 nonzeros  0s
Presolve : Reductions: rows 0(-1); columns 0(-14); elements 0(-7) - Reduced to empty
Solving the original LP from the solution after postsolve
Model status        : Optimal
Objective value     :  7.6600000000e+04
Relative P-D gap    :  0.0000000000e+00
HiGHS run time      :          0.00
Optimal objective value: $76600.0

Generator dispatch (MW):
G3: 40.0
G5: 30.0
G4: 30.0
G1: 350.0
G2: 50.0
G6: 0.0

Wind turbine dispatch (MW):
T1: 1000.0

Wind curtailment (MW):
T1: 0.0


In [40]:
# Validate your solution
@assert objective_value(dispatchModel) == 76600 "Objective value should be between 76600"

# 2. Solving scenarios of the ED problem

We now want to solve all sceanrios. To do so we wrap the model in a
function that we then can call with different inputs.

> **Note**
>
> Copy your model into the function. The results should be stored in the
> dataframe.

In [43]:
# Create a function `solve_economic_dispatch`, which solves the economic
# dispatch problem for a given set of input parameters.

function solve_economic_dispatch(dictThermalGeneartor::Dict, dictWindTurbines::Dict, scenario)
    ## Define the economic dispatch (ED) model
    dispatchModel = Model(HiGHS.Optimizer)
    set_silent(dispatchModel)
    ## Define decision variables
    ## p_g power output of generators - short form syntax
    @variable(dispatchModel, dictThermalGeneartor[g].min <= p_g[g in keys(dictThermalGeneartor)] <= dictThermalGeneartor[g].max)
    
    ## p_w wind power injection - short form syntax
    @variable(dispatchModel, 0 <= p_w[w in keys(dictWindTurbines)] <= scenario.wind_forecast)

    ## Define the objective function 
    @objective(
        dispatchModel,
        Min,
        sum(dictThermalGeneartor[g].variable_cost * p_g[g] for g in keys(dictThermalGeneartor)) +
        sum(dictWindTurbines[w].variable_cost * p_w[w] for w in keys(dictWindTurbines))
    )
  
    ## Define the power balance constraint
    @constraint(
        dispatchModel,
        sum(p_g[g] for g in keys(dictThermalGeneartor)) + 
        sum(p_w[w] for w in keys(dictWindTurbines)) == scenario.demand_forecast
    )
    
    ## Solve statement
    optimize!(dispatchModel)
    assert_is_solved_and_feasible(dispatchModel)

    ## return the optimal value of the objective function and variables
    return (
        p_g = value.(p_g),
        p_w = value.(p_w),
        wind_curtailment = scenario.wind_forecast - sum(value.(p_w)),
        total_cost = objective_value(dispatchModel),
    )
end

# Create a dataframe to store results
results_df = DataFrame(
    scenario = String[],
    total_cost = Float64[],
    wind_curtailment = Float64[]
)

# Loop over the scenarios and save the results to a dataframe
for (scenario_name, scenario_data) in dictScenarios
    solution = solve_economic_dispatch(dictThermalGeneartor, dictWindTurbines, scenario_data)
    push!(results_df, (scenario_name, solution.total_cost, solution.wind_curtailment))
end

# Print the dataframe
println("\nResults for all scenarios:")
println(results_df)



Results for all scenarios:
5×3 DataFrame
 Row │ scenario  total_cost  wind_curtailment 
     │ String    Float64     Float64          
─────┼────────────────────────────────────────
   1 │ S4           66600.0               0.0
   2 │ S5           51600.0             250.0
   3 │ S1           76600.0               0.0
   4 │ S3           71600.0               0.0
   5 │ S2           81600.0              50.0


What is the problem in scenario “S5” with the assumptions made in the ED
problem leading to an inefficient usage of wind turbines?

In [0]:
# YOUR ANSWER HERE
# In scenario "S5", there are two main issues:
# 1. The wind forecast is very high (2000 MW) but the demand is relatively low (1000 MW). 
#    The economic dispatch model treats wind power like any other generator and curtails the excess wind 
#    power since there is no way to store it. This leads to significant wind curtailment.
# 2. Due to the minimum power constraints of thermal generators (lower bounds of p_g), about 250 MW 
#    of thermal generation must always run even though it's more expensive than wind power. This 
#    forces us to curtail even more wind power and leads to higher system costs than necessary.
# A more sophisticated model could include energy storage, consider selling excess power to neighboring
# regions, or allow for complete shutdown of thermal units (unit commitment) to make better use of the available wind resources.
# This will be covered in the next lectures. The unit commitement problem with multiple timesteps and storages is introduced.



------------------------------------------------------------------------

# Solutions

You will likely find solutions to most exercises online. However, I
strongly encourage you to work on these exercises independently without
searching explicitly for the exact answers to the exercises.
Understanding someone else’s solution is very different from developing
your own. Use the lecture notes and try to solve the exercises on your
own. This approach will significantly enhance your learning and
problem-solving skills.

Remember, the goal is not just to complete the exercises, but to
understand the concepts and improve your programming abilities. If you
encounter difficulties, review the lecture materials, experiment with
different approaches, and don’t hesitate to ask for clarification during
class discussions.

Later, you will find the solutions to these exercises online in the
associated GitHub repository, but we will also quickly go over them in
next week’s tutorial. To access the solutions, click on the Github
button on the lower right and search for the folder with today’s lecture
and tutorial. Alternatively, you can ask ChatGPT or Claude to explain
them to you. But please remember, the goal is not just to complete the
exercises, but to understand the concepts and improve your programming
abilities.